In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import nltk
import nltk.sentiment
import re

from requests import get
from bs4 import BeautifulSoup
import os

import acquire
import prepare

In [2]:
#acquire.acquire_repo_list('petroleum')

In [3]:
#acquire.acquire_repo_list('solar energy')

In [4]:
#acquire.acquire_repo_list('natural gas')

In [5]:
#df = acquire.scrape_github_data()

In [6]:
df = pd.read_csv('all_repos_dict.csv')

In [7]:
df

,repo,language,readme_contents
0,ColasGael/Machine-Learning-for-Solar-Energy-Pr...,Python,# Machine-Learning-for-Solar-Energy-Prediction...
1,meltaxa/solariot,Python,# Solariot\n\nLeverage your IoT enabled Solar ...
2,renewables-ninja/gsee,Python,[![Master branch build status](https://img.shi...
3,3KUdelta/Solar_WiFi_Weather_Station,C++,# Zambretti Forecaster on Solar WiFi Weather S...
4,ropensci/nasapower,R,\n[![R build status](https://github.com/ropens...
...,...,...,...
295,agregoire/qc-drilling-timeline,JavaScript,No README
296,chrord/Data_driven_Distributed_Operation_of_El...,NaN,# Data-driven Distributed Operation of Electri...
297,chaiitanyasangani88/OperationalFlowRateVariation,MATLAB,# OperationalFlowCaseStudy\nA detailed study a...
298,jaburney/naturalgastransition,R,## Coal-to-Natural Gas Transition Impacts Anal...


In [8]:
df.loc[:100, 'category'] = 'solar_energy'

In [9]:
df.loc[100:200, 'category'] = 'petroleum'

In [10]:
df.loc[200:, 'category'] = 'natural_gas'

In [11]:
df = prepare.prep_repo_data(df)

In [12]:
df

,language,category,repo,readme_contents,clean_tokes,clean_stemmed,clean_lemmatized
0,Python,solar_energy,ColasGael/Machine-Learning-for-Solar-Energy-Pr...,# Machine-Learning-for-Solar-Energy-Prediction...,"[machinelearningforsolarenergyprediction, by, ...",machinelearningforsolarenergypredict adel kuzm...,machinelearningforsolarenergyprediction adele ...
1,Python,solar_energy,meltaxa/solariot,# Solariot\n\nLeverage your IoT enabled Solar ...,"[solariot, leverage, your, iot, enabled, solar...",solariot leverag iot enabl solar pv invert str...,solariot leverage iot enabled solar pv inverte...
2,Python,solar_energy,renewables-ninja/gsee,[![Master branch build status](https://img.shi...,"[master, branch, build, statushttpsimgshieldsi...",master branch build statushttpsimgshieldsioazu...,master branch build statushttpsimgshieldsioazu...
3,C++,solar_energy,3KUdelta/Solar_WiFi_Weather_Station,# Zambretti Forecaster on Solar WiFi Weather S...,"[zambretti, forecaster, on, solar, wifi, weath...",zambretti forecast solar wifi weather station ...,zambretti forecaster solar wifi weather statio...
4,R,solar_energy,ropensci/nasapower,\n[![R build status](https://github.com/ropens...,"[r, build, statushttpsgithubcomropenscinasapow...",r build statushttpsgithubcomropenscinasapowerw...,r build statushttpsgithubcomropenscinasapowerw...
...,...,...,...,...,...,...,...
294,Python,natural_gas,fernandessfae/Evolucao-da-producao-de-gas-natu...,# Projeto DataScience - Evolução da produção d...,"[projeto, datascience, evoluo, da, produo, de,...",projeto datasci evoluo da produo de gs natur p...,projeto datascience evoluo da produo de g natu...
296,NaN,natural_gas,chrord/Data_driven_Distributed_Operation_of_El...,# Data-driven Distributed Operation of Electri...,"[datadriven, distributed, operation, of, elect...",datadriven distribut oper electr natur ga syst...,datadriven distributed operation electricity n...
297,MATLAB,natural_gas,chaiitanyasangani88/OperationalFlowRateVariation,# OperationalFlowCaseStudy\nA detailed study a...,"[operationalflowcasestudy, a, detailed, study,...",operationalflowcasestudi detail studi oper flo...,operationalflowcasestudy detailed study operat...
298,R,natural_gas,jaburney/naturalgastransition,## Coal-to-Natural Gas Transition Impacts Anal...,"[coaltonatural, gas, transition, impacts, anal...",coaltonatur ga transit impact analysi thi repo...,coaltonatural gas transition impact analysis r...


In [13]:
df.shape

(241, 7)

In [14]:
df.reset_index(drop=True)

,language,category,repo,readme_contents,clean_tokes,clean_stemmed,clean_lemmatized
0,Python,solar_energy,ColasGael/Machine-Learning-for-Solar-Energy-Pr...,# Machine-Learning-for-Solar-Energy-Prediction...,"[machinelearningforsolarenergyprediction, by, ...",machinelearningforsolarenergypredict adel kuzm...,machinelearningforsolarenergyprediction adele ...
1,Python,solar_energy,meltaxa/solariot,# Solariot\n\nLeverage your IoT enabled Solar ...,"[solariot, leverage, your, iot, enabled, solar...",solariot leverag iot enabl solar pv invert str...,solariot leverage iot enabled solar pv inverte...
2,Python,solar_energy,renewables-ninja/gsee,[![Master branch build status](https://img.shi...,"[master, branch, build, statushttpsimgshieldsi...",master branch build statushttpsimgshieldsioazu...,master branch build statushttpsimgshieldsioazu...
3,C++,solar_energy,3KUdelta/Solar_WiFi_Weather_Station,# Zambretti Forecaster on Solar WiFi Weather S...,"[zambretti, forecaster, on, solar, wifi, weath...",zambretti forecast solar wifi weather station ...,zambretti forecaster solar wifi weather statio...
4,R,solar_energy,ropensci/nasapower,\n[![R build status](https://github.com/ropens...,"[r, build, statushttpsgithubcomropenscinasapow...",r build statushttpsgithubcomropenscinasapowerw...,r build statushttpsgithubcomropenscinasapowerw...
...,...,...,...,...,...,...,...
236,Python,natural_gas,fernandessfae/Evolucao-da-producao-de-gas-natu...,# Projeto DataScience - Evolução da produção d...,"[projeto, datascience, evoluo, da, produo, de,...",projeto datasci evoluo da produo de gs natur p...,projeto datascience evoluo da produo de g natu...
237,NaN,natural_gas,chrord/Data_driven_Distributed_Operation_of_El...,# Data-driven Distributed Operation of Electri...,"[datadriven, distributed, operation, of, elect...",datadriven distribut oper electr natur ga syst...,datadriven distributed operation electricity n...
238,MATLAB,natural_gas,chaiitanyasangani88/OperationalFlowRateVariation,# OperationalFlowCaseStudy\nA detailed study a...,"[operationalflowcasestudy, a, detailed, study,...",operationalflowcasestudi detail studi oper flo...,operationalflowcasestudy detailed study operat...
239,R,natural_gas,jaburney/naturalgastransition,## Coal-to-Natural Gas Transition Impacts Anal...,"[coaltonatural, gas, transition, impacts, anal...",coaltonatur ga transit impact analysi thi repo...,coaltonatural gas transition impact analysis r...


In [19]:
df.dropna(inplace=True)

In [ ]:
df.he

In [ ]:
plt.figure(figsize=(16, 9))
pd.Series(df.clean_tokes).value_counts().head(12).plot.barh(width=.9, ec='black', title='12 most common words')
plt.xlabel('Number of occurances')

In [ ]:
df.clean_lemmatized

In [ ]:
#ADDITIONAL_STOPWORDS = ['r', 'u', '2', 'ltgt']

def clean(text: str) -> list:
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    text = (text.encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split() # tokenization
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

all_words = clean(' '.join(df.clean_stemmed))

plt.figure(figsize=(16, 9))
pd.Series(all_words).value_counts().head(12).plot.barh(width=.9, ec='black', title='12 most common words')
plt.xlabel('Number of occurances')

In [ ]:
df.clean_stemmed.str.len()